#Startup funding data cleaning

In [ ]:
import pandas as pd
import re
import numpy as np


In [3]:
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

### Importing raw dataset

In [50]:
df=pd.read_csv("startup_funding (RAW) - Copy.csv")
df

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09-01-2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13-01-2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09-01-2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02-01-2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02-01-2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN
...,...,...,...,...,...,...,...,...,...,...
3039,3040,29-01-2015,Printvenue,NaN,NaN,NaN,Asia Pacific Internet Group,Private Equity,"45,00,000",NaN
3040,3041,29-01-2015,Graphene,NaN,NaN,NaN,KARSEMVEN Fund,Private Equity,"8,25,000",Govt backed VC Fund
3041,3042,30-01-2015,Mad Street Den,NaN,NaN,NaN,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000",NaN
3042,3043,30-01-2015,Simplotel,NaN,NaN,NaN,MakeMyTrip,Private Equity,NaN,"Strategic Funding, Minority stake"


#### Renaming the Columns

In [ ]:
print(df.columns)

Index(['Sr No', 'Date dd/mm/yyyy', 'Startup Name', 'Industry Vertical',
       'SubVertical', 'City  Location', 'Investors Name', 'InvestmentnType',
       'Amount in USD', 'Remarks'],
      dtype='object')

In [52]:
df.rename(columns = {
                'Date dd/mm/yyyy' : 'Date', 
                'SubVertical' : 'Sub-Vertical', 
               'City  Location' : 'City',
               'InvestmentnType' : 'Investment Type',
               'Amount in USD' : 'Amount (USD)'
},
inplace=True)
df.columns

Index(['Sr No', 'Date', 'Startup Name', 'Industry Vertical', 'Sub-Vertical',
       'City', 'Investors Name', 'Investment Type', 'Amount (USD)', 'Remarks'],
      dtype='object')

In [53]:
df

,Sr No,Date,Startup Name,Industry Vertical,Sub-Vertical,City,Investors Name,Investment Type,Amount (USD),Remarks
0,1,09-01-2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13-01-2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09-01-2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02-01-2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02-01-2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN
...,...,...,...,...,...,...,...,...,...,...
3039,3040,29-01-2015,Printvenue,NaN,NaN,NaN,Asia Pacific Internet Group,Private Equity,"45,00,000",NaN
3040,3041,29-01-2015,Graphene,NaN,NaN,NaN,KARSEMVEN Fund,Private Equity,"8,25,000",Govt backed VC Fund
3041,3042,30-01-2015,Mad Street Den,NaN,NaN,NaN,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000",NaN
3042,3043,30-01-2015,Simplotel,NaN,NaN,NaN,MakeMyTrip,Private Equity,NaN,"Strategic Funding, Minority stake"


#### Decoding encoded sequnece  (like \\xc2\\xa0)

In [54]:
def has_special_chars(text):
    if not isinstance(text, str):
        return False
    
    return bool(re.search(r'[^\w\s.,;:!?"\'()&@#\/-]',text))

special_char_rows = {}

text_cols = df.select_dtypes(include='object').columns

for col in text_cols:
    mask = df[col].apply(has_special_chars)
    if mask.any():
        special_char_rows[col] = df[mask][col]

for col, rows in special_char_rows.items():
    print(f"\n column:{col}")
    print(rows.to_string(index=False))


 column:Date
\\xc2\\xa010/7/2015

 column:Startup Name
                                      BYJU’S
                                  "BYJU\\'S"
                What\\xe2\\x80\\x99s Up Life
                        Byju\\xe2\\x80\\x99s
             Creator\\xe2\\x80\\x99s Gurukul
                        Byju\\xe2\\x80\\x99s
                        SERV\\xe2\\x80\\x99D
                        BYJU\\xe2\\x80\\x99s
                        Byju\\xe2\\x80\\x99s
                       \\xc2\\xa0CloudCherry
                     Let\xe2\x80\x99s Barter
       Don\\xe2\\x80\\x99t Scratch Your Head
                        Byju\\xe2\\x80\\x99s
                             Alef\\nMobitech
                        PlanMy\\nMedicalTrip
                     Carl\\xe2\\x80\\x99s Jr
NearBuy (previously\\xc2\\xa0 groupon India)
                    \\xc2\\xa0News in shorts
                         \\xc2\\xa0Bluestone
                          \\xc2\\xa0Shopsity
                          \\xc2\\xa0Notesgen

In [55]:
import pandas as pd
import re
import html

# Function to clean text
def clean_text(text):
    if pd.isna(text):
        return text
    # Decode escape sequences
    text = bytes(text, "utf-8").decode("unicode_escape", errors="ignore")
    # Remove residual unicode characters
    text = text.encode("latin1", errors="ignore").decode("utf-8", errors="ignore")
    # Replace multiple spaces/newlines with single space
    text = re.sub(r'\s+', ' ', text)
    # Remove leading/trailing spaces and quotes
    text = text.strip(" '\"")
    text = re.sub(r'\(adsbygoogle\s*=\s*window\.adsbygoogle\s*\|\|\s*\[\]\)\.push\(\{\}\);', '', text)
    text = re.sub(r'\+', '', text)
    return text



In [56]:
columns_to_clean = ['Date', 'Startup Name', 'Industry Vertical', 'Sub-Vertical',
       'City', 'Investors Name', 'Investment Type', 'Amount (USD)', 'Remarks']
for _ in range(2):
       for col in columns_to_clean:
              df[col] = df[col].astype(str).apply(clean_text)

In [57]:
def has_special_chars(text):
    if not isinstance(text, str):
        return False
    
    return bool(re.search(r'[^\w\s.,;:!?"\'()&@#\/-]',text))

special_char_rows = {}

text_cols = df.select_dtypes(include='object').columns

for col in text_cols:
    mask = df[col].apply(has_special_chars)
    if mask.any():
        special_char_rows[col] = df[mask][col]

for col, rows in special_char_rows.items():
    print(f"\n column:{col}")
    print(rows.to_string(index=False))


 column:Startup Name
                 BYJU’S
         What’s Up Life
                 Byju’s
      Creator’s Gurukul
                 Byju’s
                 SERV’D
                 BYJU’s
                 Byju’s
           Let’s Barter
Don’t Scratch Your Head
                 Byju’s
              Carl’s Jr
                Faaso’s

 column:Industry Vertical
WiFi – first Cloud communication platform
    ECommerce Brands’ Full Service Agency
                      Chain of Tea Café’s

 column:Sub-Vertical
parenting blog and kids’ events discovery platform
                   Women’s Fashion Apparel eTailer
                Online Women’s Fashion Marketplace
                    Men’s Grooming Product etailer
                           Women’s Fashion etailer
                Online Women’s Fashion Marketplace
                   Children’s Entertainment Portal
                  Women’s Fashion ECommerce Portal
         Children’s Entertainment Games & Products
                            Wome

#### Cleaning with null values

In [58]:
df

,Sr No,Date,Startup Name,Industry Vertical,Sub-Vertical,City,Investors Name,Investment Type,Amount (USD),Remarks
0,1,09-01-2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",nan
1,2,13-01-2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",nan
2,3,09-01-2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",nan
3,4,02-01-2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",nan
4,5,02-01-2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",nan
...,...,...,...,...,...,...,...,...,...,...
3039,3040,29-01-2015,Printvenue,nan,nan,nan,Asia Pacific Internet Group,Private Equity,"45,00,000",nan
3040,3041,29-01-2015,Graphene,nan,nan,nan,KARSEMVEN Fund,Private Equity,"8,25,000",Govt backed VC Fund
3041,3042,30-01-2015,Mad Street Den,nan,nan,nan,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000",nan
3042,3043,30-01-2015,Simplotel,nan,nan,nan,MakeMyTrip,Private Equity,nan,"Strategic Funding, Minority stake"


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Sr No              3044 non-null   int64 
 1   Date               3044 non-null   object
 2   Startup Name       3044 non-null   object
 3   Industry Vertical  3044 non-null   object
 4   Sub-Vertical       3044 non-null   object
 5   City               3044 non-null   object
 6   Investors Name     3044 non-null   object
 7   Investment Type    3044 non-null   object
 8   Amount (USD)       3044 non-null   object
 9   Remarks            3044 non-null   object
dtypes: int64(1), object(9)
memory usage: 237.9+ KB


### Cleaning date column

In [61]:
df["Date"].unique()

array(['09-01-2020', '13-01-2020', '02-01-2020', ..., '29-01-2015',
       '30-01-2015', '31-01-2015'], shape=(1035,), dtype=object)

In [62]:
pattern = r'^\d{2}-\d{2}-\d{4}$'
invalid_dates=df[~df['Date'].str.match(pattern, na=False)]['Date'].unique()
print(invalid_dates)

['05/072018' '01/07/015' '10/7/2015' '12/05.2015' '13/04.2015'
 '15/01.2015' '22/01//2015']


In [63]:
df['Date'] = df['Date'].str.replace('/','-',regex=False)\
                                            .str.replace('.','-',regex=False)\
                                            .str.replace('--','-',regex=False)
pattern = r'^\d{2}-\d{2}-\d{4}$'
invalid_dates=df[~df['Date'].str.match(pattern, na=False)]['Date'].unique()
print(invalid_dates)

['05-072018' '01-07-015' '10-7-2015']


In [64]:
df['Date'] = df['Date'].str.replace(r'(\d{2})-(\d{2})(\d{4})',r'\1-\2-\3',regex=True)
pattern = r'^\d{2}-\d{2}-\d{4}$'
invalid_dates=df[~df['Date'].str.match(pattern, na=False)]['Date'].unique()
print(invalid_dates)

['01-07-015' '10-7-2015']


In [66]:
def fix_short_year(date_str):
    if isinstance(date_str,str) and re.fullmatch(r'\d{2}-\d{2}-\d{3}',date_str):
        parts = date_str.split('-')
        day,month,year = parts
        if len(year) == 3:
            return f"{day}-{month}-2{year[-3:]}"
    return date_str
df=df.applymap(fix_short_year)
pattern = r'^\d{2}-\d{2}-\d{4}$'
invalid_dates=df[~df['Date'].str.match(pattern, na=False)]['Date'].unique()
print(invalid_dates)

['10-7-2015']


C:\Users\ankit\AppData\Local\Temp\ipykernel_12280\1895661104.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df=df.applymap(fix_short_year)


In [67]:

def clean_and_format_date(date_str):
    if not isinstance(date_str,str):
        return date_str
    
    cleaned = re.sub(r'^[^0-9]*','',date_str)
    match = re.search(r'(\d{1,2})[-\/](\d{1,2})[-\/](\d{4})', cleaned)
    if match:
        day = match.group(1).zfill(2)
        month = match.group(2).zfill(2)
        year = match.group(3)
    return f"{day}-{month}-{year}"

df['Date'] = df['Date'].apply(clean_and_format_date)

In [68]:
pattern = r'^\d{2}-\d{2}-\d{4}$'
invalid_dates=df[~df['Date'].str.match(pattern, na=False)]['Date'].unique()
print(invalid_dates)

[]


#### Changing data type of date column to datetime

In [69]:
df['Date'] = pd.to_datetime(df['Date'],format='%d-%m-%Y')
df.dtypes

Sr No                         int64
Date                 datetime64[ns]
Startup Name                 object
Industry Vertical            object
Sub-Vertical                 object
City                         object
Investors Name               object
Investment Type              object
Amount (USD)                 object
Remarks                      object
dtype: object

### Cleaning Startup Name column

In [70]:
special_char=df[df['Startup Name'].str.contains(r'[^a-zA-Z0-9\s]',regex=True)]
print(special_char['Startup Name'])

0                             BYJU’S
3       https://www.wealthbucket.in/
16                      Lenskart.com
19      Sunstone Eduversity Pvt. Ltd
53                         Lo! Foods
                    ...             
3031                 Englishleap.com
3033                   Caratlane.com
3037                    CarDekho.com
3038                   Grabhouse.com
3043                couponmachine.in
Name: Startup Name, Length: 152, dtype: object


In [206]:
df['Startup Name']=df['Startup Name']\
    .str.replace(r'\.com','',regex=True)\
    .str.replace(r'\.in','',regex=True)\
    .str.replace(r'\.co','',regex=True)\
    .str.replace(r'Pvt. Ltd','',regex=True)\
    .str.replace(r'\.bio','',regex=True)\
    .str.replace(r'https://www.','',regex=True)\
    .str.replace(r"\\'","'",regex=True)\
    .str.replace(r'#','',regex=True)\
    .str.replace(r'/','',regex=True)\
    .str.replace(r'"','',regex=True)\
    .str.replace(r"'","’",regex=True)\
    .str.replace(r'\.','',regex=True)\
    .str.replace(r'’',"'",regex = True)\
    .str.replace(r'\[.*?\]|\(.*?\)|\{.?\}','',regex=True).str.strip()
special_char=df[df['Startup Name'].str.contains(r'[^a-zA-Z0-9\s]',regex=True)]
print(special_char['Startup Name'])

0                                              BYJU'S
53                                          Lo! Foods
67                                             BYJU'S
118     Samunnati Financial Intermediation & Services
130                                   A&R Bon Vivants
138                       Aavishkaar-Intellecap Group
239                                    What's Up Life
712                                            Byju's
713                                 Creator's Gurukul
796                                            i-lend
983                                            Byju's
1153                                           SERV'D
1164                                           BYJU's
1347                                       CheersOye!
1356                                           Byju's
1398                                     Let's Barter
1514                          Don't Scratch Your Head
1880                                           Byju's
2282                        

### Cleaning Industry Vertical Column

In [72]:
df['Industry Vertical']

0                    E-Tech
1            Transportation
2                E-commerce
3                   FinTech
4       Fashion and Apparel
               ...         
3039                    nan
3040                    nan
3041                    nan
3042                    nan
3043                    nan
Name: Industry Vertical, Length: 3044, dtype: object

In [73]:
df['Industry Vertical'] = df['Industry Vertical'].replace(r'(?i)^nan$', np.nan, regex = True)

In [74]:
df['Industry Vertical'].isnull().sum()

np.int64(171)

In [ ]:
df['Industry Vertical'].value_counts

<bound method IndexOpsMixin.value_counts of 0                    E-Tech
1            Transportation
2                E-commerce
3                   FinTech
4       Fashion and Apparel
               ...         
3039                    NaN
3040                    NaN
3041                    NaN
3042                    NaN
3043                    NaN
Name: Industry Vertical, Length: 3044, dtype: object>

### Training an ML model to fill NA values in Industry Vertical column


In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

df_train = df[df['Industry Vertical'].notna()]
df_predict = df[df['Industry Vertical'].isna()]

features = ["Startup Name", "Industry Vertical", "Sub-Vertical","City", "Investment Type", "Amount (USD)"]
target = "Industry Vertical"

imputer = SimpleImputer(strategy='most_frequent')
df_train[features] = imputer.fit_transform(df_train[features])
df_predict[features] = imputer.transform(df_predict[features])

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
x_train = encoder.fit_transform(df_train[features])
x_predict = encoder.transform(df_predict[features])

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train[target])

model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)

predicted = model.predict(x_predict)
predicted_labels = label_encoder.inverse_transform(predicted)

df.loc[df['Industry Vertical'].isna(), 'Industry Vertical'] = predicted_labels


C:\Users\ankit\AppData\Local\Temp\ipykernel_12280\2472196287.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[features] = imputer.fit_transform(df_train[features])
C:\Users\ankit\AppData\Local\Temp\ipykernel_12280\2472196287.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict[features] = imputer.transform(df_predict[features])


In [79]:
df['Industry Vertical']

0                    E-Tech
1            Transportation
2                E-commerce
3                   FinTech
4       Fashion and Apparel
               ...         
3039      Consumer Internet
3040      Consumer Internet
3041      Consumer Internet
3042      Consumer Internet
3043      Consumer Internet
Name: Industry Vertical, Length: 3044, dtype: object

In [80]:
df

,Sr No,Date,Startup Name,Industry Vertical,Sub-Vertical,City,Investors Name,Investment Type,Amount (USD),Remarks
0,1,2020-01-09,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",nan
1,2,2020-01-13,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",nan
2,3,2020-01-09,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",nan
3,4,2020-01-02,wealthbucket,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",nan
4,5,2020-01-02,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",nan
...,...,...,...,...,...,...,...,...,...,...
3039,3040,2015-01-29,Printvenue,Consumer Internet,nan,nan,Asia Pacific Internet Group,Private Equity,"45,00,000",nan
3040,3041,2015-01-29,Graphene,Consumer Internet,nan,nan,KARSEMVEN Fund,Private Equity,"8,25,000",Govt backed VC Fund
3041,3042,2015-01-30,Mad Street Den,Consumer Internet,nan,nan,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000",nan
3042,3043,2015-01-30,Simplotel,Consumer Internet,nan,nan,MakeMyTrip,Private Equity,nan,"Strategic Funding, Minority stake"


In [81]:
df['Industry Vertical'].isnull().sum()

np.int64(0)

#### Cleaning Sub-Vertical Column

In [83]:
df['Sub-Vertical'] = df['Sub-Vertical'].replace(r'(?i)^nan$', np.nan, regex = True)

In [84]:
df.isnull().sum()

Sr No                  0
Date                   0
Startup Name           0
Industry Vertical      0
Sub-Vertical         936
City                   0
Investors Name         0
Investment Type        0
Amount (USD)           0
Remarks                0
dtype: int64

### Training an ML model to fill NA values in Sub-Vertical column


In [86]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

df_train = df[df['Sub-Vertical'].notna()]
df_predict = df[df['Sub-Vertical'].isna()]

features = ["Startup Name", "Industry Vertical", "Sub-Vertical","City", "Investment Type", "Amount (USD)"]
target = "Sub-Vertical"

imputer = SimpleImputer(strategy='most_frequent')
df_train[features] = imputer.fit_transform(df_train[features])
df_predict[features] = imputer.transform(df_predict[features])

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
x_train = encoder.fit_transform(df_train[features])
x_predict = encoder.transform(df_predict[features])

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train[target])

model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)

predicted = model.predict(x_predict)
predicted_labels = label_encoder.inverse_transform(predicted)

df.loc[df['Sub-Vertical'].isna(), 'Sub-Vertical'] = predicted_labels


C:\Users\ankit\AppData\Local\Temp\ipykernel_12280\3439127576.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[features] = imputer.fit_transform(df_train[features])
C:\Users\ankit\AppData\Local\Temp\ipykernel_12280\3439127576.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict[features] = imputer.transform(df_predict[features])


In [87]:
df['Sub-Vertical']

0                                  E-learning
1                   App based shuttle service
2       Retailer of baby and toddler products
3                           Online Investment
4                 Embroiled Clothes For Women
                        ...                  
3039                  Online Lending Platform
3040                  Online Lending Platform
3041                  Online Lending Platform
3042                  Online Lending Platform
3043                  Online Lending Platform
Name: Sub-Vertical, Length: 3044, dtype: object

In [88]:
df.isnull().sum()

Sr No                0
Date                 0
Startup Name         0
Industry Vertical    0
Sub-Vertical         0
City                 0
Investors Name       0
Investment Type      0
Amount (USD)         0
Remarks              0
dtype: int64

In [89]:
df['Sub-Vertical'].unique()

array(['E-learning', 'App based shuttle service',
       'Retailer of baby and toddler products', ...,
       'Coding Skill Showcase & Resume Builder',
       'Hobby & Activity Class Aggregator', 'Interior Design Marketplace'],
      shape=(1942,), dtype=object)

#### Cleaning City Column

In [93]:
df['City'] = df['City'].replace(r'(?i)^nan$', np.nan, regex = True)

In [94]:
df.isnull().sum()

Sr No                  0
Date                   0
Startup Name           0
Industry Vertical      0
Sub-Vertical           0
City                 180
Investors Name         0
Investment Type        0
Amount (USD)           0
Remarks                0
dtype: int64

### Training an ML model to fill NA values in City column


In [95]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

df_train = df[df['City'].notna()]
df_predict = df[df['City'].isna()]

features = ["Startup Name", "Industry Vertical", "Sub-Vertical","City", "Investment Type", "Amount (USD)"]
target = "City"

imputer = SimpleImputer(strategy='most_frequent')
df_train[features] = imputer.fit_transform(df_train[features])
df_predict[features] = imputer.transform(df_predict[features])

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
x_train = encoder.fit_transform(df_train[features])
x_predict = encoder.transform(df_predict[features])

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train[target])

model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)

predicted = model.predict(x_predict)
predicted_labels = label_encoder.inverse_transform(predicted)

df.loc[df['City'].isna(), 'City'] = predicted_labels


C:\Users\ankit\AppData\Local\Temp\ipykernel_12280\4167061211.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[features] = imputer.fit_transform(df_train[features])
C:\Users\ankit\AppData\Local\Temp\ipykernel_12280\4167061211.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict[features] = imputer.transform(df_predict[features])


In [96]:
df['City']

0       Bengaluru
1         Gurgaon
2       Bengaluru
3       New Delhi
4          Mumbai
          ...    
3039    Bangalore
3040    Bangalore
3041    Bangalore
3042    Bangalore
3043    Bangalore
Name: City, Length: 3044, dtype: object

#### Correct names of Cities in City column

In [154]:
df['City'].unique()

array(['Bengaluru', 'Gurugram', 'New Delhi', 'Mumbai', 'Chennai', 'Pune',
       'Noida', 'Faridabad', 'San Francisco', 'San Jose', 'Amritsar',
       'Koramangala', 'Tulangan', 'Hyderabad', 'Burnsville', 'Menlo Park',
       'Palo Alto', 'Santa Monica', 'Singapore', 'Tharamani', 'Andheri',
       'Chembur', 'Nairobi', 'Haryana', 'New York', 'Karnataka', 'Bhopal',
       'Jaipur', 'Nagpur', 'Indore', 'California', 'Ahmedabad',
       'Rourkela', 'Srinagar', 'Bhubaneswar', 'Chandigarh', 'Kolkata',
       'Coimbatore', 'Udaipur', 'Surat', 'Panaji', 'lucknow', 'Gaya',
       'Vadodara', 'Thiruvananthapuram', 'Missouri', 'Gwalior', 'Karur',
       'Udupi', 'Kochi', 'Agra', 'Hubballi', 'Kozhikode', 'Siliguri',
       'Lucknow', 'Kanpur', 'London', 'Varanasi', 'Jodhpur', 'Boston',
       'Belagavi'], dtype=object)

In [157]:
df['City'] = df['City'].replace({'Gurgaon' : 'Gurugram', 
                                           'Delhi': 'New Delhi',
                                           'Kormangala' : 'Koramangala',
                                           'Taramani' : 'Tharamani',
                                           'Mumbai/Bengaluru' : 'Mumbai',
                                           'Bengaluru and Gurugram' : 'Bengaluru',
                                           'India/Singapore' : 'New Delhi',
                                           'India/US' : 'New York',
                                           'New York, Bengaluru' : 'Bengaluru',
                                           'India' : 'New Delhi',
                                           'Delhi & Cambridge' : 'New Delhi',
                                           'Ahemadabad' : 'Ahmedabad',
                                           'Kolkatta' : 'Kolkata',
                                           'Bangalore' : 'Bengaluru',
                                           'Ahemdabad' : 'Ahmedabad',
                                           'Bhubneswar' : 'Bhubaneswar',
                                           'Nw Delhi' : 'New Delhi',
                                           'Trivandrum' : 'Thiruvananthapuram',
                                           'Missourie' : 'Missouri',
                                           'Bangalore/ Bangkok' : 'Bengaluru',
                                           'Hubli' : 'Hubballi',
                                           'Kerala' : 'Thiruvananthapuram',
                                           'Goa' : 'Panaji',
                                           'Uttar Pradesh' : 'Lucknow',
                                           'US' : 'New York',
                                           'USA' : 'New York',
                                           'SFO / Bangalore' : 'Bengaluru',
                                           'Seattle / Bangalore' : 'Bengaluru',
                                           'Pune/Seattle' : 'Pune',
                                           'Pune / Dubai' : 'Pune',
                                           'Bangalore / SFO' : 'Bengaluru',
                                           'New Delhi / US' : 'New Delhi',
                                           'Mumbai / UK' : 'Mumbai',
                                            'Hyderabad/USA' : 'Hyderabad',
                                            'Bangalore / Palo Alto' : 'Bengaluru',
                                            'Mumbai / NY': 'Mumbai',
                                            'USA/India' : 'New Delhi',
                                            'Goa/Hyderabad' : 'Panaji',
                                            'Noida / Singapore' : 'Noida',
                                            'Belgaum' : 'Belagavi',
                                            'Pune / US' : 'Pune',
                                            'Chennai/ Singapore' : 'Chennai',
                                            'Pune / Singapore' : 'Pune',
                                            'Bangalore / San Mateo' : 'Bengaluru',
                                            'New York/ India' : 'New York',
                                            'US/India' : 'New Delhi',
                                            'Gurgaon / SFO' : 'Gurugram',
                                            'Bangalore / USA' : 'Bengaluru',
                                            'New Delhi/ Houston' : 'New Delhi',
                                            'Mumbai / Global' : 'Mumbai',
                                            'India / US' : 'New Delhi',
                                            'New Delhi / California' : 'New Delhi',
                                            'Dallas / Hyderabad' : 'Hyderabad',
                                            'San Jose,' : 'San Jose',
                                            'lucknow' : 'Lucknow'
                                           })
df['City'].unique()

array(['Bengaluru', 'Gurugram', 'New Delhi', 'Mumbai', 'Chennai', 'Pune',
       'Noida', 'Faridabad', 'San Francisco', 'San Jose', 'Amritsar',
       'Koramangala', 'Tulangan', 'Hyderabad', 'Burnsville', 'Menlo Park',
       'Palo Alto', 'Santa Monica', 'Singapore', 'Tharamani', 'Andheri',
       'Chembur', 'Nairobi', 'Haryana', 'New York', 'Karnataka', 'Bhopal',
       'Jaipur', 'Nagpur', 'Indore', 'California', 'Ahmedabad',
       'Rourkela', 'Srinagar', 'Bhubaneswar', 'Chandigarh', 'Kolkata',
       'Coimbatore', 'Udaipur', 'Surat', 'Panaji', 'Lucknow', 'Gaya',
       'Vadodara', 'Thiruvananthapuram', 'Missouri', 'Gwalior', 'Karur',
       'Udupi', 'Kochi', 'Agra', 'Hubballi', 'Kozhikode', 'Siliguri',
       'Kanpur', 'London', 'Varanasi', 'Jodhpur', 'Boston', 'Belagavi'],
      dtype=object)

In [158]:
df

,Sr No,Date,Startup Name,Industry Vertical,Sub-Vertical,City,Investors Name,Investment Type,Amount (USD),Remarks
0,1,2020-01-09,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",nan
1,2,2020-01-13,Shuttl,Transportation,App based shuttle service,Gurugram,Susquehanna Growth Equity,Series C,"80,48,394",nan
2,3,2020-01-09,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",nan
3,4,2020-01-02,wealthbucket,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",nan
4,5,2020-01-02,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",nan
...,...,...,...,...,...,...,...,...,...,...
3039,3040,2015-01-29,Printvenue,Consumer Internet,Online Lending Platform,Bengaluru,Asia Pacific Internet Group,Private Equity,"45,00,000",nan
3040,3041,2015-01-29,Graphene,Consumer Internet,Online Lending Platform,Bengaluru,KARSEMVEN Fund,Private Equity,"8,25,000",Govt backed VC Fund
3041,3042,2015-01-30,Mad Street Den,Consumer Internet,Online Lending Platform,Bengaluru,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000",nan
3042,3043,2015-01-30,Simplotel,Consumer Internet,Online Lending Platform,Bengaluru,MakeMyTrip,Private Equity,nan,"Strategic Funding, Minority stake"


#### Cleaning Investor Name Column

In [165]:
df['Investors Name'] = df['Investors Name'].replace(r'^nan$',np.nan,regex = True)

In [166]:
df.iloc[13]

Sr No                                           14
Date                           2019-12-16 00:00:00
Startup Name                                InCred
Industry Vertical                          Finance
Sub-Vertical         Non-Banking Financial Company
City                                        Mumbai
Investors Name                                 NaN
Investment Type                       Debt Funding
Amount (USD)                             59,00,000
Remarks                                        nan
Name: 13, dtype: object

In [167]:
df.isnull().sum()

Sr No                 0
Date                  0
Startup Name          0
Industry Vertical     0
Sub-Vertical          0
City                  0
Investors Name       24
Investment Type       0
Amount (USD)          0
Remarks               0
dtype: int64

In [169]:
df['Investors Name'] = df['Investors Name'].fillna('Unknown')

In [171]:
df.isnull().sum()

Sr No                0
Date                 0
Startup Name         0
Industry Vertical    0
Sub-Vertical         0
City                 0
Investors Name       0
Investment Type      0
Amount (USD)         0
Remarks              0
dtype: int64

#### Cleaning Investment Type Column

In [172]:
df['Investment Type'] = df['Investment Type'].replace(r'nan$',np.nan,regex = True)

In [173]:
df.isnull().sum()

Sr No                0
Date                 0
Startup Name         0
Industry Vertical    0
Sub-Vertical         0
City                 0
Investors Name       0
Investment Type      4
Amount (USD)         0
Remarks              0
dtype: int64

In [174]:
df['Investment Type'].unique()

array(['Private Equity Round', 'Series C', 'Series B', 'Pre-series A',
       'Seed Round', 'Series A', 'Series D', 'Seed', 'Series F',
       'Series E', 'Debt Funding', 'Series G', 'Series H', 'Venture',
       'Seed Funding', nan, 'Funding Round', 'Corporate Round',
       'Maiden Round', 'pre-series A', 'Seed Funding Round',
       'Single Venture', 'Venture Round', 'Pre-Series A', 'Angel',
       'Series J', 'Angel Round', 'pre-Series A',
       'Venture - Series Unknown', 'Bridge Round', 'Private Equity',
       'Debt and Preference capital', 'Inhouse Funding',
       'Seed/ Angel Funding', 'Debt', 'Pre Series A', 'Equity',
       'Debt-Funding', 'Mezzanine', 'Series B (Extension)',
       'Equity Based Funding', 'Private Funding', 'Seed / Angel Funding',
       'Seed/Angel Funding', 'Seed funding', 'Seed / Angle Funding',
       'Angel / Seed Funding', 'Private', 'Structured Debt', 'Term Loan',
       'PrivateEquity', 'Angel Funding', 'Crowd funding', 'Crowd Funding'],
      dty

In [177]:
df['Investment Type'].mode()

0    Seed Funding
Name: Investment Type, dtype: object

In [175]:
df['Investment Type'] = df['Investment Type'].fillna(df['Investment Type'].mode()[0])

In [178]:
df.isnull().sum()

Sr No                0
Date                 0
Startup Name         0
Industry Vertical    0
Sub-Vertical         0
City                 0
Investors Name       0
Investment Type      0
Amount (USD)         0
Remarks              0
dtype: int64

#### Cleaning Amount (USD) column

In [179]:
df['Amount (USD)'] = df['Amount (USD)'].replace(r'nan$',np.nan,regex=True)

In [180]:
df.isnull().sum()

Sr No                  0
Date                   0
Startup Name           0
Industry Vertical      0
Sub-Vertical           0
City                   0
Investors Name         0
Investment Type        0
Amount (USD)         960
Remarks                0
dtype: int64

In [182]:
print(df['Amount (USD)'].describe())

count          2084
unique          471
top       10,00,000
freq            165
Name: Amount (USD), dtype: object


In [ ]:
df['Amount (USD)'] = df['Amount (USD)'].str.replace(',','')
df['Amount (USD)'] = pd.to_numeric(df['Amount (USD)'], errors='coerce')

In [193]:
median_amount = df['Amount (USD)'].median()
df['Amount (USD)'].fillna(median_amount,inplace = True)

In [197]:
df['Amount (USD)'].isnull().sum()

np.int64(0)

In [198]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Sr No              3044 non-null   int64         
 1   Date               3044 non-null   datetime64[ns]
 2   Startup Name       3044 non-null   object        
 3   Industry Vertical  3044 non-null   object        
 4   Sub-Vertical       3044 non-null   object        
 5   City               3044 non-null   object        
 6   Investors Name     3044 non-null   object        
 7   Investment Type    3044 non-null   object        
 8   Amount (USD)       3044 non-null   float64       
 9   Remarks            3044 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 237.9+ KB


In [199]:
df['Remarks'] = df['Remarks'].replace(r'nan$',np.nan,regex = True)

In [201]:
df.isnull().sum()

Sr No                   0
Date                    0
Startup Name            0
Industry Vertical       0
Sub-Vertical            0
City                    0
Investors Name          0
Investment Type         0
Amount (USD)            0
Remarks              2625
dtype: int64

In [202]:
df['Remarks'].fillna('No Remarks', inplace = True)

In [203]:
df.isnull().sum()

Sr No                0
Date                 0
Startup Name         0
Industry Vertical    0
Sub-Vertical         0
City                 0
Investors Name       0
Investment Type      0
Amount (USD)         0
Remarks              0
dtype: int64

In [207]:
df

,Sr No,Date,Startup Name,Industry Vertical,Sub-Vertical,City,Investors Name,Investment Type,Amount (USD),Remarks
0,1,2020-01-09,BYJU'S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,200000000.0,No Remarks
1,2,2020-01-13,Shuttl,Transportation,App based shuttle service,Gurugram,Susquehanna Growth Equity,Series C,8048394.0,No Remarks
2,3,2020-01-09,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,18358860.0,No Remarks
3,4,2020-01-02,wealthbucket,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,3000000.0,No Remarks
4,5,2020-01-02,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,1800000.0,No Remarks
...,...,...,...,...,...,...,...,...,...,...
3039,3040,2015-01-29,Printvenue,Consumer Internet,Online Lending Platform,Bengaluru,Asia Pacific Internet Group,Private Equity,4500000.0,No Remarks
3040,3041,2015-01-29,Graphene,Consumer Internet,Online Lending Platform,Bengaluru,KARSEMVEN Fund,Private Equity,825000.0,Govt backed VC Fund
3041,3042,2015-01-30,Mad Street Den,Consumer Internet,Online Lending Platform,Bengaluru,"Exfinity Fund, GrowX Ventures.",Private Equity,1500000.0,No Remarks
3042,3043,2015-01-30,Simplotel,Consumer Internet,Online Lending Platform,Bengaluru,MakeMyTrip,Private Equity,1750000.0,"Strategic Funding, Minority stake"


In [208]:
df.to_csv('cleaned_startup_data.csv')

## End of the data cleaning